## Check the whole feature maps not only last layer

How about simply modify the first layer or the layer that difference is getting bigger?

So check the whole layers' feature maps when the original image and perturbed image get into the model

In [2]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
from PGD import PGD
from my_resnet import *
from models import *
import os
import torchvision

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "2"  # Set the GPUs 1 and 2 to use
    
cuda = True if torch.cuda.is_available() else False
print(cuda)

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

batch_size = 100

test_dataset = torchvision.datasets.CIFAR10(root='/workspace/my_data', train=False, download=False, transform=transform_test)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

model = ResNet18() ## original model
checkpoint_path2 = '/workspace/checkpoint/resnet18_cifar10'
checkpoint2 = torch.load(checkpoint_path2)['net']
for key in list(checkpoint2.keys()):
    checkpoint2[key.replace('module.', '')] = checkpoint2.pop(key)
model.load_state_dict(checkpoint2, strict=False)## original model
model = model.cuda()
model.eval()

attack = PGD(model, cuda=cuda) # original model

cos = nn.CosineSimilarity(dim=2)

max_dict = {}
min_dict = {}

for ori_imgs, ori_labels in test_loader:

    pgd_imgs = attack(ori_imgs, ori_labels)
    ori_imgs = ori_imgs.cuda()

    # output_dict = {}
    
    # def forwardhooking(layer_name):
    #     def hook(m, inp, out):
    #         output_dict[layer_name] = out
    #     return hook
    holder = []
    
    def forwardhooking(m, input, output):
        holder.append(output)
    
    for name, m in model.named_modules():
        if "conv" in name or "bn" in name or "linear" in name:
            m.register_forward_hook(forwardhooking)
    
    model(ori_imgs)
    model(pgd_imgs)

torch.cuda.empty_cache()

ModuleNotFoundError: No module named 'pytorch_grad_cam'

In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
from PGD import PGD
from my_resnet import *
from models import *
import os
import torchvision

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2"  # Set the GPUs 2 and 3 to use
    
cuda = True if torch.cuda.is_available() else False
print(device)

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

batch_size = 100

test_dataset = torchvision.datasets.CIFAR10(root='/workspace/my_data', train=False, download=False, transform=transform_test)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

model = ResNet18() ## original model
checkpoint_path2 = '/workspace/checkpoint/resnet18_cifar10'
checkpoint2 = torch.load(checkpoint_path2)['net']
for key in list(checkpoint2.keys()):
    checkpoint2[key.replace('module.', '')] = checkpoint2.pop(key)
model.load_state_dict(checkpoint2, strict=False)## original model
model = model.cuda()
model.eval()


robust_model = ResNet18()
checkpoint_path1 = '/workspace/checkpoint/pgd_adversarial_training'
checkpoint1 = torch.load(checkpoint_path1)['net']
for key in list(checkpoint1.keys()):
    checkpoint1[key.replace('module.', '')] = checkpoint1.pop(key)
robust_model.load_state_dict(checkpoint1, strict=False)
robust_model = robust_model.cuda()
robust_model.eval()

attack = PGD(model, cuda=cuda) # original model

cos = nn.CosineSimilarity(dim=2)

max_dict = {}
min_dict = {}

for ori_imgs, ori_labels in test_loader:

    pgd_imgs = attack(ori_imgs, ori_labels)
    ori_imgs = ori_imgs.cuda()
    # robust_pgd_imgs = attack1(ori_imgs, ori_labels)
    # pgd_input_tensor = attack(ori_imgs, ori_labels)
    # Note: input_tensor can be a batch tensor with several images!

    holder = []
            
    def forwardhooking(m, inp, out):
        holder.append(out)
    # forwardhooking = ForwardHooking

    for name, m in robust_model.named_modules():
        if name == 'layer4.1':
            m.register_forward_hook(forwardhooking)

    robust_model(ori_imgs)
    robust_model(pgd_imgs)
    

    # get cosine similarity along 512channel in last layer
    sim = cos(holder[0][:,:,-1], holder[1][:,:,-1])
    max_values, max_indcies = torch.topk(sim, 10, dim=1)
    max_values = max_values.tolist()
    max_indcies = max_indcies.squeeze(0).tolist()
    min_values, min_indcies = torch.topk(sim, 10, dim=1, largest=False)
    min_values = min_values.tolist()
    min_indcies = min_indcies.squeeze(0).tolist()

    for i in range(batch_size):
        for j in range(10):
            if max_indcies[i][j] in max_dict:
                max_dict[max_indcies[i][j]].append(max_values[i][j])
            else:
                max_dict[max_indcies[i][j]] = [max_values[i][j]]
                
            if min_indcies[i][j] in min_dict:
                min_dict[min_indcies[i][j]].append(min_values[i][j])
            else:
                min_dict[min_indcies[i][j]] = [min_values[i][j]]
    


min_dict_sorted = sorted(min_dict.items(), key=lambda x: x[1], reverse=False)
max_dict_sorted = sorted(max_dict.items(), key=lambda x: x[1], reverse=True)
torch.cuda.empty_cache()

ModuleNotFoundError: No module named 'pytorch_grad_cam'